In [48]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product
import random
import math
import pickle
from tqdm.notebook import tqdm_notebook
import sys
import torch
import torch.nn as nn
sys.path.append('../')
import fn
from addition_dataset import GroupAddition

In [2]:
%load_ext autoreload
%autoreload 2

## Testing

In [180]:
# Specify base and carry table
b = 4
table = 1*(np.add.outer(np.arange(b),np.arange(b))>=b)

In [194]:
# Create dataset and dataloader
num_passes = 10000
dataset = GroupAddition(table, 2, ids=None, interleaved=True, digit_order='reversed')
dataset = torch.utils.data.ConcatDataset([dataset] * num_passes)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [195]:
# Define model
class LSTMModel(nn.Module):
    '''Simple LSTM model for testing purposes
    '''
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        '''Initialize model with specified parameters
        '''
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, X):
        '''Return forward-pass including missing values of X
        '''
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, X.size(0), self.hidden_size).to(X.device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(X, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out

In [217]:
class HoldoutLoss(nn.Module):
    def __init__(self, ):
        super().__init__()

    def forward(self, X_out, s, ids):
        MSE = nn.MSELoss()
        import pdb; pdb.set_trace()
        loss = MSE(X_out[:,ids], s)
        return loss

In [218]:
model = LSTMModel(b, b, 1, b)

In [219]:
# Define criterion and optimizer
criterion = HoldoutLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [220]:
# Train data set
t = 0
for batch_idx, (X, s, ids) in enumerate(dataloader):
    
    # Compute and print loss
    loss = criterion(model(X), s.float(), ids)
    if t % 100 == 0:
        print('t =', t, 'loss =', loss.item())
    
    # Zero gradients, perform a backward pass, and update the weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Iterate counter
    t += 1

> /var/folders/qd/lngl0tnd4557ml_18l05dm8r0000gn/T/ipykernel_15174/2238870521.py(8)forward()
      5     def forward(self, X_out, s, ids):
      6         MSE = nn.MSELoss()
      7         import pdb; pdb.set_trace()
----> 8         loss = MSE(X_out[:,ids], s)
      9         return loss



ipdb>  X_out


tensor([[ 0.4059, -0.2827, -0.4261, -0.2886],
        [ 0.4010, -0.3101, -0.4599, -0.2736],
        [ 0.4016, -0.3047, -0.4509, -0.2835],
        [ 0.4004, -0.3039, -0.4567, -0.2730],
        [ 0.4027, -0.3025, -0.4459, -0.2834],
        [ 0.3996, -0.2996, -0.4560, -0.2769],
        [ 0.4092, -0.2774, -0.4126, -0.2897],
        [ 0.4114, -0.2655, -0.3940, -0.3027],
        [ 0.4041, -0.2895, -0.4344, -0.2944],
        [ 0.4043, -0.2916, -0.4308, -0.2850],
        [ 0.3991, -0.3016, -0.4593, -0.2770],
        [ 0.4088, -0.2767, -0.4132, -0.2902],
        [ 0.4041, -0.2866, -0.4336, -0.2815],
        [ 0.4100, -0.2843, -0.4114, -0.2909],
        [ 0.4098, -0.2742, -0.4083, -0.2906],
        [ 0.4133, -0.2615, -0.3851, -0.3027]], grad_fn=<AddmmBackward0>)


ipdb>  X_out.shape


torch.Size([16, 4])


ipdb>  q


Need to learn more about LSTM's to figure out correct implementation.